In [8]:
import pandas as pd
from functools import reduce
import numpy as np
# Load the Excel file
file_path = "output_data/Project_Data.xlsx"
excel_file = pd.ExcelFile(file_path)



In [12]:
# loading each sheet into a DataFrame
sheets_dict = {sheet_name: pd.read_excel(excel_file, sheet_name=sheet_name) for sheet_name in excel_file.sheet_names}

# Load the Excel workbook
workbook = pd.ExcelFile(file_path)


# Columns to delete from each sheet
columns_to_delete = ['Status', 'Age', 'His/Lat Ethnicity', 'Fed Race Description', 'Home Language Description', 'Econ Dis', 'Qualifies as Eco Dis', 'Class', 'Withdrw', 'Ent_y']


In [2]:
# Load the Excel workbook
workbook = pd.ExcelFile(file_path)

# Clean each sheet by removing specified columns and collect cleaned dataframes
cleaned_sheets = {}
for sheet_name in workbook.sheet_names:
    df = pd.read_excel(workbook, sheet_name=sheet_name)
    
    # Remove columns that are present in the dataframe and are to be deleted
    cols_to_remove = [col for col in columns_to_delete if col in df.columns]
    df_cleaned = df.drop(columns=cols_to_remove, errors='ignore')
    
    # Store the cleaned dataframe
    cleaned_sheets[sheet_name] = df_cleaned

# Create a list of dataframes to merge
dfs_to_merge = [df for df in cleaned_sheets.values()]

# Perform a multi-DataFrame merge using reduce and merge
combined_df = reduce(lambda left, right: pd.merge(left, right, on='Other ID', how='outer'), dfs_to_merge)

# Display the combined dataframe info to verify the merge and structure


In [13]:
# Explicitly convert any non-standard missing values to NaN
# Assuming 'None' or some other markers might be present
combined_df = combined_df.replace([None, ''], np.nan)
combined_df.head()

,Ent_x,Cal Code_x,Other ID,Sex,Local Race,Grad Date_x,DOB,Birth Country,Student Language Description,Marital Status,...,Sub,Class Status,Earned Cred,Cat,Description,Grade,Ent,Cal Code_y,Grad Date_y,Withdrawal Date
0,1,001,837,F,WHITE/NON HISPANIC,NaT,1992-01-17,Iraq,ARABIC,Married,...,15,Drp,0.0,R,ESL Wrkshp SS,09,NaN,NaN,NaT,NaT
1,1,001,837,F,WHITE/NON HISPANIC,NaT,1992-01-17,Iraq,ARABIC,Married,...,15,Drp,0.0,R,ESL Wrkshp Wrt,09,NaN,NaN,NaT,NaT
2,1,001,837,F,WHITE/NON HISPANIC,NaT,1992-01-17,Iraq,ARABIC,Married,...,07,Drp,0.5,R,Health,09,NaN,NaN,NaT,NaT
3,1,001,837,F,WHITE/NON HISPANIC,NaT,1992-01-17,Iraq,ARABIC,Married,...,14,Drp,0.0,R,MAPS A,09,NaN,NaN,NaT,NaT
4,1,001,837,F,WHITE/NON HISPANIC,NaT,1992-01-17,Iraq,ARABIC,Married,...,14,Drp,0.5,R,Newcomer ELD A1,09,NaN,NaN,NaT,NaT


In [14]:
output_file_path = "output_data/New_Project_Data.xlsx"
combined_df.to_excel(output_file_path, index=False)

In [17]:
 #Grouping by 'Other ID' and aggregating the data
aggregated_df = combined_df.groupby('Other ID').agg({
    'Earned Cred': 'sum',  # Sum of credits earned
    'DOB': 'min',  # Earliest date of birth
    'Sex': lambda x: x.mode()[0] if not x.mode().empty else None,
    'Marital Status': lambda x: x.mode()[0] if not x.mode().empty else None,
    # Add other columns as needed
    'Grad Date_x': 'max',  # Latest graduation date
    # Add other columns as needed
}).reset_index()

aggregated_df.head()

,Other ID,Earned Cred,DOB,Sex,Marital Status,Grad Date_x
0,0,0.0,2000-07-29,F,None,NaT
1,1,128.0,1977-11-11,F,Married,2017-11-04
2,2,1.5,1988-05-20,M,None,NaT
3,3,88.0,1978-04-08,F,Single,2019-05-30
4,4,98.0,1983-07-03,F,Single,2017-06-03
